In [20]:
import pandas as pd, numpy as np, os, re
from bs4 import BeautifulSoup
from glob import iglob
import codecs


In [254]:
from sklearn import linear_model

#### Rename html files

In [100]:
path_al = '/home/jeferson/personal_projects/scraping_cars/results/al'
files = os.listdir(path_al)


for file in files:
    os.rename(os.path.join(path, file), os.path.join(path, file.split('-')[1] + '.html'))

In [49]:
# Alabama : 724716247 , 706708719

def soup_func(filename):
    html = codecs.open(path_al + '/' + filename, 'r').read()
    return BeautifulSoup(html, 'lxml')



In [104]:
def get_price(soup):
    vehicle_price = soup.find('span', class_=
              'vehicle-info__price-display').text
    return vehicle_price

In [106]:
def get_msrp_price(soup):
    vehicle_msrp_tag = soup.find('span',
        attrs={'class':'vehicle-info__price-label vehicle-info__price-label--msrp'})
    if vehicle_msrp_tag:
        sibling_msrp = vehicle_msrp_tag.nextSibling
        if sibling_msrp:
            return sibling_msrp.text


In [107]:
def get_price_drop(soup):
    price_drop = soup.find('span',
            attrs={'class':'vdp-cap-price__prev vehicle-info__price-drop'})

    if price_drop:
        tag_drop = price_drop.find('s', class_='strike-through')
        if tag_drop:
            return tag_drop.text

In [108]:
def price_comparison_tool(soup):
    badge_tag = soup.find('a', attrs={'cars-smooth-scroll':'price-comparison-tool'} )

    if badge_tag:
        return badge_tag.find('span').text

In [199]:
def dealer_rating(soup):
    dealer_ratings = soup.find('p', class_='rating__link rating__link-details' )
    if dealer_ratings:
        list_rate = re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", dealer_ratings.text)
        if len(list_rate) > 1:
            return [list_rate[0], list_rate[1]]
        else:
            return [-1, -1]
    

In [146]:
def overall_rating(soup):
    overall_rating = soup.find('cars-star-ratings' )
    if overall_rating:
        return overall_rating.get('bound-rating')

In [144]:
def consumers_review(soup):
    consumer_reviews = soup.find('div', class_='review-stars-average' )
    if consumer_reviews:
        a_lnk = consumer_reviews.find('a', attrs={'data-linkname':'consumer-reviews-total-reviews'})
        if a_lnk:
            return a_lnk.text



In [152]:
def mileage(soup):
    mileage = soup.find('div', class_='vdp-cap-price__mileage--mobile vehicle-info__mileage')
    if mileage:
        return mileage.text

In [154]:
def count_photo(soup):
    count_photos = soup.find('div', class_='photo-count__count')
    if count_photos:
        return count_photos.text    
    

In [173]:
s = '(4.8) 1086 Reviews'

In [175]:
list_rate = re.findall(r"[+-]? *(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?", s)
print(list_rate[0], list_rate[1])

4.8  1086


In [193]:
soup = soup_func('724716247.html')
print(f)
print(get_price(soup))
print(get_msrp_price(soup))
print(get_price_drop(soup))
print(price_comparison_tool(soup))

# print(dealer_rating(soup))
list_rate_seller = dealer_rating(soup)
seller_rating, seller_count_rating = list_rate_seller[0], list_rate_seller[1]
print('seller_rate', seller_rating , '\n', 'seller_count_review', seller_count_rating)
# print('\n', list_rate_seller, '\n')

print(overall_rating(soup))
print(consumers_review(soup))
print(mileage(soup))
print(count_photo(soup))
print('\n\n')

765918626.html
$32,802
$39,015
$33802
None
seller_rate 4.6 
 seller_count_review 262
4.9
115 reviews
26 miles
12





In [189]:
list_rate_seller

'4.9'

In [185]:
list_rate

'4.9'

In [203]:
tab = []
for f in files[:1400]:
    soup = soup_func(f)
    
    list_rate_seller = dealer_rating(soup)   
    seller_rating, seller_count_rating = list_rate_seller[0], list_rate_seller[1]

    tab.append([f, get_price(soup), get_msrp_price(soup), get_price_drop(soup),
              price_comparison_tool(soup), seller_rating, seller_count_rating, overall_rating(soup),
              consumers_review(soup), mileage(soup), count_photo(soup)])


In [204]:
df_al = pd.DataFrame(tab, columns=['id','price', 'msrp_price', 'original_price', 
                                  'good_offer', 'seller_rating', 'seller_count_rating', 'car_rate', 'consumers_review',
                                  'mileage', 'n_photos'])

df_al.to_csv('1400.csv', sep=';', index=False)

df_al_bkp = df_al.copy()


In [207]:
df_al.price = df_al.price.apply(lambda x: x.replace('$', '').replace(',', ''))
df_al.msrp_price = df_al.msrp_price.apply(lambda x: x.replace('$', '').replace(',', '') if x else x)

df_al.original_price = df_al.original_price.apply(lambda x: x.replace('$', '').replace(',', '') if x else x)
df_al.mileage = df_al.mileage.apply(lambda x: x.replace('miles', '').strip() if x else x)

In [218]:
df_al.seller_rating = df_al.seller_rating.astype('float')
df_al.car_rate = df_al.car_rate.astype('float')

df_al.n_photos = df_al.n_photos.apply(lambda x: int(x) if x else x)
df_al.seller_count_rating = df_al.seller_count_rating.astype('int')


In [222]:
df_al.consumers_review = df_al.consumers_review.apply(lambda x: int(x.replace('reviews', '').strip()) if x else x)

In [224]:
df_al.mileage = df_al.mileage.apply(lambda x: float(x.replace(',', '').strip()) if x else x) 

In [229]:
df_al.drop('id', inplace=True, axis=1)

In [234]:
df_al.fillna(-1, inplace=True)

In [236]:
df_al.price = df_al.price.astype('float')
df_al.msrp_price = df_al.msrp_price.astype('float')
df_al.original_price = df_al.original_price.astype('float')

In [238]:
# df_al_bkp_2 = df_al.copy()

# df_al = df_al_bkp_2.copy()

In [248]:
df_al = df_al.merge(pd.get_dummies(df_al.good_offer), how='left', right_index=True, left_index=True)

In [249]:
df_al.drop('good_offer', inplace=True, axis=1)

In [251]:
df_al.drop(-1, inplace=True, axis=1)

In [252]:
df_al

,price,msrp_price,original_price,seller_rating,seller_count_rating,car_rate,consumers_review,mileage,n_photos,Fair Price,Good Deal,Great Deal
0,23665.0,23665.0,-1.0,4.6,122,4.3,129.0,10.0,15.0,0,0,0
1,28994.0,-1.0,-1.0,4.8,1086,4.8,36.0,37224.0,41.0,0,1,0
2,31000.0,34515.0,-1.0,4.9,226,5.0,12.0,5.0,32.0,0,0,0
3,19471.0,20230.0,-1.0,5.0,169,4.4,45.0,10.0,18.0,0,0,0
4,22995.0,-1.0,23995.0,4.8,1086,4.8,54.0,51225.0,43.0,0,1,0
5,28419.0,29980.0,-1.0,4.9,287,4.7,25.0,1.0,9.0,0,0,0
6,22574.0,24120.0,-1.0,4.8,236,4.9,55.0,14.0,18.0,0,0,0
7,33000.0,-1.0,33800.0,4.6,289,4.7,97.0,50180.0,29.0,0,0,1
8,17300.0,-1.0,-1.0,4.6,263,4.8,550.0,75639.0,35.0,0,1,0
9,35285.0,-1.0,-1.0,4.5,63,4.6,81.0,89029.0,32.0,0,1,0


### Models

In [259]:
X = df_al.drop('price', axis=1)
y = df_al['price']


lm = linear_model.LinearRegression()
model = lm.fit(X,y)

In [261]:
predictions = lm.predict(X)


In [263]:
predictions[:10]

array([31211.06378962, 28114.50797872, 39274.72567791, 31407.73086817,
       27417.67881816, 34690.57571662, 32795.81897126, 22498.38460673,
       12570.46812304, 18885.22429016])

In [264]:
lm.score(X,y)

0.3745876732123202